In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pymongo
import warnings
import re
warnings.filterwarnings("ignore")
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")
tags_dictionary = client.get(data_as_key)[-1]['tags']
len(tags_dictionary)

6484

In [2]:
bert_data_key = ("ids_to_validate", "data", "bert_tags")
guten_tag_labels = client.get(bert_data_key)[-1]["gutenTag"]    

In [5]:
from collections import Counter

In [4]:
#Исключить из предсказаний энтити фиксированные тэги

In [11]:
endDate = datetime(2020, 5, 20)
startDate = datetime.today() 
excess_from_models_counter = 0
excess_from_person_counter = 0
excess_from_person_gutentag_counter = 0
excess_from_person_entity_counter = 0
total_number_of_tags = 0
excess_from_models_gutentag_counter = 0
excess_from_models_entity_counter = 0
validator_remove_collector = []
validator_add_collector = []

validator_remove_BERT_collector = []
validator_remove_ENTITY_collector = []




EntityBERT_collector_add = []
EntityNoNBERT_collector_add = []
EntityBERT_collector_add = []
BERT_collector_add = []
BothModels_add = []

EntityBERT_collector_remove = []
EntityNoNBERT_collector_remove = []
EntityBERT_collector_remove = []
BERT_collector_remove = []
BothModels_remove = []
Proper_tags_collector = []

totalProcessedValidator = db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"}, {"status":"graphicone_feed"},
                                                                          {"status":"validated"}],
                                                                "entity_prepared": {"$exists": True},
                                                                "entity_response_aws_2": {"$exists": True},
                                      'validated_timestamp': {'$gte': endDate, '$lte': startDate}
                                     })

for i in totalProcessedValidator:
    bert_tags = set(i["bert_tags"])
    entity_prepared = set(i["entity_prepared"])
    confirmed_after_validate = set([row.lower() for row in i["grafeed_confirmed"]])    
    Proper_tags_collector += confirmed_after_validate
    
    EntityBERT_collector_remove_1 = (set(entity_prepared).intersection(set(guten_tag_labels))) - (set(confirmed_after_validate).intersection(set(guten_tag_labels)))        
    
    
    EntityNoNBERT_collector_remove_1 = (set(entity_prepared) - set(guten_tag_labels)) - (set(confirmed_after_validate) - set(guten_tag_labels))
    EntityNoNBERT_collector_add_1 = (set(confirmed_after_validate) - set(guten_tag_labels)) - (set(entity_prepared) - set(guten_tag_labels))
    
    BERT_collector_add_1 = (set(confirmed_after_validate).intersection(set(guten_tag_labels))) - set(bert_tags)
    BERT_collector_remove_1 = set(bert_tags) - (set(confirmed_after_validate).intersection(set(guten_tag_labels)))
    
    EntityBERT_collector_remove += EntityBERT_collector_remove_1
    EntityNoNBERT_collector_add += EntityNoNBERT_collector_add_1
    EntityNoNBERT_collector_remove += EntityNoNBERT_collector_remove_1
    BERT_collector_add += BERT_collector_add_1
    BERT_collector_remove += BERT_collector_remove_1

#     Both_Models_Collector_add = 
    
    
#     print("EntityBERT_collector_add_1:",EntityBERT_collector_add_1)    
    print()
    print("bert_tags", bert_tags)
    print("entity_prepared:",entity_prepared)
    print("confirmed_after_validate:",confirmed_after_validate)
    print()


    print("EntityBERT_collector_remove_1:", EntityBERT_collector_remove_1)    
    print("EntityNoNBERT_collector_add_1", EntityNoNBERT_collector_add_1)
    print("EntityNoNBERT_collector_remove_1", EntityNoNBERT_collector_remove_1)
    print("BERT_collector_add_1", BERT_collector_add_1)
    print("BERT_collector_remove_1", BERT_collector_remove_1)
#     validator_remove_BERT_collector += (set(bert_tags) - (set(confirmed_after_validate).intersection(set(guten_tag_labels))))
#     validator_remove_ENTITY_collector += (set(entity_prepared) - set(confirmed_after_validate))
#     print(i["entity_response_aws_2"])
#     print(i["validator_username"])
#     print("bert_tags:",bert_tags)
    print()
    
#     excess_from_models = ((bert_tags.union(entity_prepared))- confirmed_after_validate)
#     excess_from_models_counter += len(excess_from_models)    
#     print("validator_remove",excess_from_models)
#     validator_remove_collector += list(excess_from_models)
    
#     excess_from_models_gutentag = excess_from_models.intersection(set(guten_tag_labels))
#     excess_from_models_gutentag_counter+= len(excess_from_models_gutentag)    
#     excess_from_models_entity = excess_from_models - (set(guten_tag_labels))
#     excess_from_models_entity_counter+= len(excess_from_models_entity)

    
    
#     excess_from_person = (confirmed_after_validate - (bert_tags.union(entity_prepared)))
#     excess_from_person_counter += len(excess_from_person)
    
#     print("validator_add",excess_from_person)
#     validator_add_collector += list(excess_from_person)
    
#     excess_from_person_gutentag = excess_from_person.intersection(set(guten_tag_labels))
#     excess_from_person_gutentag_counter+= len(excess_from_person_gutentag)
    
#     excess_from_person_entity = excess_from_person - (set(guten_tag_labels))
#     excess_from_person_entity_counter+= len(excess_from_person_entity)

    
#     print()
#     print()
#     print()
#     print()


bert_tags {'mining', 'commodities', 'disease', 'metals'}
entity_prepared: {'metals', 'health', 'global', 'world health organization', 'mining', 'commodities', 'copper'}
confirmed_after_validate: {'covid-19', 'disease', 'metals', 'health', 'coronavirus', 'pandemic', 'global', 'world health organization', 'mining', 'commodities', 'copper'}

EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 {'pandemic', 'covid-19', 'coronavirus'}
EntityNoNBERT_collector_remove_1 set()
BERT_collector_add_1 {'health', 'global'}
BERT_collector_remove_1 set()


bert_tags {'global', 'funds'}
entity_prepared: {'global', 'funds', 'idm'}
confirmed_after_validate: {'global', 'emerging markets', 'developed markets', 'funds'}

EntityBERT_collector_remove_1: set()
EntityNoNBERT_collector_add_1 set()
EntityNoNBERT_collector_remove_1 {'idm'}
BERT_collector_add_1 {'emerging markets', 'developed markets'}
BERT_collector_remove_1 set()


bert_tags {'macro'}
entity_prepared: {'ism', 'index', 'pmi', 'fed'}

In [10]:
total_number_of_tags = sum(dict(Counter(Proper_tags_collector)).values())
print("Proper_tags_collector", total_number_of_tags)
print("-------------------------")
print("EntityBERT_collector_remove",sum(dict(Counter(EntityBERT_collector_remove)).values()), round(sum(dict(Counter(EntityBERT_collector_remove)).values())/total_number_of_tags,3)) 
print("EntityNoNBERT_collector_add", sum(dict(Counter(EntityNoNBERT_collector_add)).values()), round(sum(dict(Counter(EntityNoNBERT_collector_add)).values())/total_number_of_tags,3)) 
print("EntityNoNBERT_collector_remove", sum(dict(Counter(EntityNoNBERT_collector_remove)).values()),round(sum(dict(Counter(EntityNoNBERT_collector_remove)).values())/total_number_of_tags,3)) 
print("BERT_collector_add", sum(dict(Counter(BERT_collector_add)).values()), round(sum(dict(Counter(BERT_collector_add)).values())/total_number_of_tags,3))
print("BERT_collector_remove", sum(dict(Counter(BERT_collector_remove)).values()),round(sum(dict(Counter(BERT_collector_remove)).values())/total_number_of_tags,3))
print("-------------------------")

Proper_tags_collector 79
-------------------------
EntityBERT_collector_remove 1 0.013
EntityNoNBERT_collector_add 15 0.19
EntityNoNBERT_collector_remove 7 0.089
BERT_collector_add 9 0.114
BERT_collector_remove 1 0.013
-------------------------


In [129]:
0.029+0.126+0.114+(0.175+0.049)

0.493

In [137]:
(0.159 + 0.079)

0.238

In [138]:
0.124+0.123

0.247

In [ ]:
# алгоритмы похожести прочекать

In [139]:
pd.DataFrame.from_dict(Counter(EntityNoNBERT_collector_add).most_common()).to_excel("ahaha2.xlsx")

In [ ]:
EntityBERT_collector_remove 152 0.032
EntityNoNBERT_collector_add 580 0.124
EntityNoNBERT_collector_remove 577 0.123
BERT_collector_add 746 0.159
BERT_collector_remove 369 0.079

#23 процента ошибок берта
#24 процента ошибок энтити